- 파라미터 개수가 많은 큰 모델이 작은 모델을 가르치는 개념
- 큰 모델의 예측과 작은 모델의 예측의 오차와 작은 모델의 손실 함수를 줄여 나가는 방향으로 작은 모델의 파라미터를 최적화

In [1]:
import tensorflow as tf
import numpy as np
from google.colab.patches import cv2_imshow
from tqdm import tqdm

In [2]:
# @title 파라미터 설정
t_epoch = 5 # @param {type:"slider", min:1, max:100, step:1}
s_epoch = 10 # @param {type:"slider", min:1, max:100, step:1}
learning_rate = 0.01
batch_size = 64 # @param {32, 64, 128, 256}{type:'raw'}
temperature = 3 # @param {type: 'slider', min:1, max:10, step:1}
alpha = 0.5 # @param {type: 'slider', min:0.1, max:0.9, step:0.1}

- @파라미터 설정 시 옵션을 바로 변경하여 적용 가능

In [3]:
# mnist 데이터셋 가져오기

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = np.reshape(x_train,(-1, 28, 28, 1))

x_test = x_test.astype('float32') / 255.
x_test = np.reshape(x_test,(-1, 28, 28, 1))

11493376/11490434 [==============================] - 0s 0us/step


- 배치 사이즈 축 추가

In [4]:
# teacher 모델
i = tf.keras.Input(shape=(28, 28, 1))
out = tf.keras.layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same')(i)
out = tf.keras.layers.LeakyReLU(alpha=0.2)(out)
out = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(out)
out = tf.keras.layers.Conv2D(512, (3, 3), strides=(2, 2), padding='same')(out)
out = tf.keras.layers.Flatten()(out)
out = tf.keras.layers.Dense(10)(out)
t_model = tf.keras.Model(inputs=[i], outputs=[out])

t_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 256)       2560      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 256)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 512)         1180160   
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                250890

- 약 140만개의 파라미터

In [5]:
# student 모델
i = tf.keras.Input(shape=(28, 28, 1))
out = tf.keras.layers.Flatten()(i)
out = tf.keras.layers.Dense(28)(out)
out = tf.keras.layers.Dense(10)(out)

s_model_1 = tf.keras.Model(inputs=[i], outputs=[out])
s_model_2 = tf.keras.models.clone_model(s_model_1)

s_model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 28)                21980     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                290       
Total params: 22,270
Trainable params: 22,270
Non-trainable params: 0
_________________________________________________________________


- Dense 레이어 2개로 구성된 단순한 student 모델
- 성능 비교를 위해 모델 하나 복제

In [6]:
# teacher 모델
t_model.compile(tf.keras.optimizers.Adam(learning_rate),
                tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# student 모델 (distilation 적용)
s_model_1.compile(tf.keras.optimizers.Adam(learning_rate),
                tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# 비교 모델 (distilation 미적용)
s_model_2.compile(tf.keras.optimizers.Adam(learning_rate),
                tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [7]:
# teacher 모델
t_model.fit(x_train, y_train, batch_size=batch_size, epochs=t_epoch)

Epoch 1/5
938/938 [==============================] - 21s 5ms/step - loss: 6.2865 - sparse_categorical_accuracy: 0.9087
Epoch 2/5
938/938 [==============================] - 5s 5ms/step - loss: 6.0644 - sparse_categorical_accuracy: 0.9560
Epoch 3/5
938/938 [==============================] - 5s 5ms/step - loss: 3.8666 - sparse_categorical_accuracy: 0.9586
Epoch 4/5
938/938 [==============================] - 5s 5ms/step - loss: 8.2154 - sparse_categorical_accuracy: 0.9558
Epoch 5/5
938/938 [==============================] - 5s 5ms/step - loss: 9.5900 - sparse_categorical_accuracy: 0.9621


- 약 96%의 정확도

In [8]:
# student 손실 함수
s_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# distilation 손실 함수
d_loss = tf.keras.losses.KLDivergence()

- Knowledge Distilation 학습에 필요한 두 loss 정의
- KLDivergence 손실함수: 서로 다른 두 개의 확률 분포를 비교해 유사성을 측정하는 지표
    - 유사할 수록 값이 작음

In [9]:
x_train.shape

(60000, 28, 28, 1)

In [11]:
batch_count = x_train.shape[0] // batch_size # 총 배치 개수

opt = tf.keras.optimizers.Adam(learning_rate)

for e in range(s_epoch):
    for _ in range(batch_count):
        batch_num = np.random.randint(0, x_train.shape[0], size=batch_size)
        t_pred = t_model.predict(x_train[batch_num])

        with tf.GradientTape() as tape:
            s_pred_1 = s_model_1(x_train[batch_num])
            student_loss = s_loss(y_train[batch_num], s_pred_1)
            distilation_loss = d_loss(
                tf.nn.softmax(t_pred / temperature, axis=1),
                tf.nn.softmax(s_pred_1 / temperature, axis=1),
            )
            loss = alpha * student_loss + (1-alpha) * distilation_loss

        vars = s_model_1.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))

        with tf.GradientTape() as tape:
            s_pred_2 = s_model_2(x_train[batch_num])
            student_loss = s_loss(y_train[batch_num], s_pred_2)
        vars = s_model_2.trainable_variables
        grad = tape.gradient(student_loss, vars)
        opt.apply_gradients(zip(grad, vars))

    print("epoch {}".format(e))
    print("선생님께 배운 경우")
    s_model_1.evaluate(x_test, y_test)
    print('혼자 공부한 경우')
    s_model_2.evaluate(x_test, y_test)
    print('\n')

epoch 0
선생님께 배운 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.4249 - sparse_categorical_accuracy: 0.9150
혼자 공부한 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.3170 - sparse_categorical_accuracy: 0.9159


epoch 1
선생님께 배운 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.4426 - sparse_categorical_accuracy: 0.9163
혼자 공부한 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.3192 - sparse_categorical_accuracy: 0.9132


epoch 2
선생님께 배운 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.4670 - sparse_categorical_accuracy: 0.9083
혼자 공부한 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.3475 - sparse_categorical_accuracy: 0.9055


epoch 3
선생님께 배운 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.4840 - sparse_categorical_accuracy: 0.9077
혼자 공부한 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.3466 - sparse_categorical_accuracy: 0.9064


epoch 4
선생님께

- 배치별로 student loss와 distilation loss 계산
- 모델 학습에 적용하는 총 손실함수는 student loss와 distilation loss를 가중 평균
- 두 번째 epoch 부터 선생님 모델로 부터 배운 모델이 정확도가 더 높게 나옴